In [1]:
# TODO:
%load_ext autoreload
%autoreload 2
import torch
import torch.onnx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# matplotlib.use('Qt5Agg')
%matplotlib Qt5Agg
import livenet
from livenet import nets, datasets


In [26]:
downscale = (14, 14)
train_x, train_y = datasets.to_plain(*datasets.get_mnist_train(), downscale=downscale, to_odd=True, to_gray=True)



In [43]:

network = nets.create_perceptron(train_x.shape[1], 2, 2)
res = network(train_x)
network.context.regularization_l1 = 0.01
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=50)
trainer.clear_life_stat = False



LiveNet


In [44]:
network.mortal = False
trainer.step(250)
network.mortal = True
trainer.step(250)



Iˈ0.000 49 0.987+0.024reg params=16 av_abs_grad 0.01 av_abs_delta: 0.01 av_abs_output 0.1 max_abs_output 0.2 max_abs_param 0.8 hcut 0 lcut 0.1 livenet/net_trainer.py:122
Iˈ0.066 99 0.952+0.026reg params=16 av_abs_grad 0.02 av_abs_delta: 0.01 av_abs_output 0.2 max_abs_output 0.2 max_abs_param 1.1 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.136 149 0.914+0.036reg params=16 av_abs_grad 0.03 av_abs_delta: 0.01 av_abs_output 0.2 max_abs_output 0.3 max_abs_param 1.5 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.206 199 0.876+0.046reg params=16 av_abs_grad 0.03 av_abs_delta: 0.01 av_abs_output 0.3 max_abs_output 0.4 max_abs_param 1.9 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.281 249 0.844+0.055reg params=16 av_abs_grad 0.04 av_abs_delta: 0.01 av_abs_output 0.3 max_abs_output 0.5 max_abs_param 2.1 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.291 N1 became useless and will die at tick 259 livenet/backend/core.py:83
Iˈ0.291 killing DestinationNeuron N1 with b=0.690, tick=259 livenet/b

In [45]:

pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y.numpy() - pred_bin
accuracy = len(diff[diff == 0]) / len(diff)
accuracy


0.7499666666666667

In [34]:

trainer.step(1)


In [2]:
from livenet import gen_utils, net_trainer
from livenet.test_livenet import _build_symmetric_dangle_net

train_x, train_y = datasets.get_odd_2()
network = _build_symmetric_dangle_net()
# train_x is not actually used as networks input doesn't connected to anything
batch_iterator = gen_utils.batch_iterator(train_x, train_y, batch_size=3)
criterion = nets.criterion_classification_n
optimizer = nets.create_optimizer(network)
trainer = net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=100)
network.context.regularization_l1 = 0.05
optimizer.learning_rate = 0.002
trainer.clear_life_stat = False
network.mortal = False
trainer.step(15000)

LiveNet
Iˈ0.000 99 15.764+0.112reg params=5 av_abs_grad 3 av_abs_delta: 0.002 av_abs_output 7 max_abs_output 7 max_abs_param 30.0 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.070 199 12.533+0.132reg params=5 av_abs_grad 3 av_abs_delta: 0.002 av_abs_output 8 max_abs_output 8 max_abs_param 29.8 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.142 299 9.148+0.151reg params=5 av_abs_grad 3 av_abs_delta: 0.002 av_abs_output 9 max_abs_output 9 max_abs_param 29.6 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.279 399 5.616+0.170reg params=5 av_abs_grad 4 av_abs_delta: 0.002 av_abs_output 9 max_abs_output 9 max_abs_param 29.4 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.420 499 2.204+0.190reg params=5 av_abs_grad 3 av_abs_delta: 0.002 av_abs_output 1e+01 max_abs_output 1e+01 max_abs_param 29.2 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.561 599 0.923+0.202reg params=5 av_abs_grad 0.1 av_abs_delta: 0.002 av_abs_output 1e+01 max_abs_output 1e+01 max_abs_param 29.0 hcut 0 lcut 0.0 livenet/ne

In [8]:

network.mortal = True
trainer.step(2000)


Iˈ0.000 15099 0.918+0.002reg params=4 av_abs_grad 0.007 av_abs_delta: 0.001 av_abs_output 1e+01 max_abs_output 1e+01 max_abs_param 21.0 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.364 15199 0.918+0.001reg params=4 av_abs_grad 0.006 av_abs_delta: 0.001 av_abs_output 1e+01 max_abs_output 1e+01 max_abs_param 21.1 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ0.725 15299 0.918+0.001reg params=4 av_abs_grad 0.006 av_abs_delta: 0.001 av_abs_output 1e+01 max_abs_output 1e+01 max_abs_param 21.1 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ1.084 15399 0.918+0.001reg params=4 av_abs_grad 0.007 av_abs_delta: 0.001 av_abs_output 1e+01 max_abs_output 1e+01 max_abs_param 21.2 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ1.445 15499 0.918+0.000reg params=4 av_abs_grad 0.007 av_abs_delta: 0.001 av_abs_output 1e+01 max_abs_output 1e+01 max_abs_param 21.3 hcut 0 lcut 0.0 livenet/net_trainer.py:122
Iˈ1.468 N0 became useless and will die at tick 15535 livenet/backend/core.py:83
Iˈ1.468 killing Destinati

In [56]:

res = network(train_x)
loss = criterion(res, train_y)


In [4]:
#livenet.utils.export_onnx(network, "/home/spometun/table/home/model.onnx")
print(f"total: {len(list(network.named_parameters()))}")
print(network.context.topology_stat.get_stat())
def name_comp_key(x: str):
    if "->" in x:
        x = "_" + x
    return x

for name, p in sorted(network.named_parameters(), key=lambda x: name_comp_key(x[0])):
    print(name, f"{p.item():.2f}")


total: 4
{'dangle': {'RegularNeuron': 1, 'DestinationNeuron': 1}, 'useless': {'RegularNeuron': 0, 'InputNeuron': 1}}
D0 -14.86
D1 -15.14
N0 21.14
N0->D1 -0.02


In [4]:
life_stat = network.context.life_stat
df = pd.DataFrame(life_stat)
df2 = df[(df["type"] == "parameter") & (df["name"] == "N0->D1")]
# df2 = df[(df["type"] == "output_low_cut_ratio") & (df["name"] == "N0")]
df2.set_index("tick")
df2["value"].plot(grid=True, x="tick", y="value", style='.-', use_index=False)


<Axes: >

In [11]:
values = df2["value"].to_list()
ticks = list(df2["tick"])

plt.ion()
plt.figure(figsize=(16, 9))
plt.grid()
plt.plot(y=values, x=ticks)
# plt.show()

TypeError: plot() got an unexpected keyword argument 'x'

In [16]:
print(df2["tick"].head())

61     0
119    1
177    2
235    3
293    4
Name: tick, dtype: int64
